In [83]:
# Uses  this API - https://punkapi.com/documentation/v2
# From  - https://hakibenita.com/fast-load-data-python-postgresql

In [72]:
import datetime
from typing import Iterator, Dict, Any
from urllib.parse import urlencode
import requests
import psycopg2

In [73]:
def parse_first_brewed(text:str) -> datetime.date:
    """
        parse dates to for correct format of datetime.date(2007,9,1)
        from ('09/2007') or ('2007')
    """
    parts = text.split('/')
    if len(parts) == 2:
        return datetime.date(int(parts[1]), int(parts[0]),1)
    elif len(parts) == 1:
        return datetime.date(int(parts[0]),1,1)
    else:
        assert False, 'Unknow date format'

In [74]:
def iter_beers_from_api(page_size: int=5) -> Iterator[Dict[str, Any]]:
    """
        Initiates session with api and pulls data
    """
    session = requests.Session()
    page = 1
    while True:
        response = session.get('https://api.punkapi.com/v2/beers?' + urlencode({
            'page': page,
            'per_page': page_size
        }))
        response.raise_for_status()
        data = response.json()
        if not data:
            break
            
        yield from data
        
        page += 1     

In [75]:
beers = iter_beers_from_api()

In [76]:
next(beers)

{'id': 1,
 'name': 'Buzz',
 'tagline': 'A Real Bitter Experience.',
 'first_brewed': '09/2007',
 'description': 'A light, crisp and bitter IPA brewed with English and American hops. A small batch brewed only once.',
 'image_url': 'https://images.punkapi.com/v2/keg.png',
 'abv': 4.5,
 'ibu': 60,
 'target_fg': 1010,
 'target_og': 1044,
 'ebc': 20,
 'srm': 10,
 'ph': 4.4,
 'attenuation_level': 75,
 'volume': {'value': 20, 'unit': 'litres'},
 'boil_volume': {'value': 25, 'unit': 'litres'},
 'method': {'mash_temp': [{'temp': {'value': 64, 'unit': 'celsius'},
    'duration': 75}],
  'fermentation': {'temp': {'value': 19, 'unit': 'celsius'}},
  'twist': None},
 'ingredients': {'malt': [{'name': 'Maris Otter Extra Pale',
    'amount': {'value': 3.3, 'unit': 'kilograms'}},
   {'name': 'Caramalt', 'amount': {'value': 0.2, 'unit': 'kilograms'}},
   {'name': 'Munich', 'amount': {'value': 0.4, 'unit': 'kilograms'}}],
  'hops': [{'name': 'Fuggles',
    'amount': {'value': 25, 'unit': 'grams'},
    '

In [77]:
connection = psycopg2.connect(
    host='localhost',
    database='testload',
    user='postgres',
    password='password@123',
)
connection.autocommit = True

In [78]:
def create_staging_table(cursor) -> None:
    cursor.execute("""
        DROP TABLE IF EXISTS staging_beers;
        CREATE UNLOGGED TABLE staging_beers (
            id INTEGER,
            name TEXT,
            tagline TEXT,
            first_brewed DATE,
            description TEXT,
            image_url TEXT,
            abv DECIMAL,
            ibu DECIMAL,
            target_fg DECIMAL,
            target_og DECIMAL,
            ebc DECIMAL,
            srm DECIMAL,
            ph DECIMAL,
            attenuation_level DECIMAL,
            brewers_tips TEXT,
            volume INTEGER
        );
    """)

In [80]:
beers = list(iter_beers_from_api())
print(f'We only have data of {len(beers)} beers') 

We only have data of 325 beers


In [81]:
def insert_one_by_one(connection, beers: Iterator[Dict[str, Any]]) -> None:
    with connection.cursor() as cursor:
        create_staging_table(cursor)
        for beer in beers:
            cursor.execute("""
                INSERT INTO staging_beers VALUES (
                    %(id)s,
                    %(name)s,
                    %(tagline)s,
                    %(first_brewed)s,
                    %(description)s,
                    %(image_url)s,
                    %(abv)s,
                    %(ibu)s,
                    %(target_fg)s,
                    %(target_og)s,
                    %(ebc)s,
                    %(srm)s,
                    %(ph)s,
                    %(attenuation_level)s,
                    %(brewers_tips)s,
                    %(volume)s
                );
            """, {
                **beer,
                'first_brewed': parse_first_brewed(beer['first_brewed']),
                'volume': beer['volume']['value']
            })

In [82]:
insert_one_by_one(connection, beers)